In [13]:
import torch
import torch.nn as nn

In [14]:
text = "Hello world! This is a simple text generation example using PyTorch."
chars = sorted(set(text))
char_to_idx = {char: i for i, char in enumerate(chars)}
idx_to_char = {i: char for i, char in enumerate(chars)}

input_text = [char_to_idx[c] for c in text]

In [15]:
class TextGenerationModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers):
        super(TextGenerationModel, self).__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.hidden = hidden_size

    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden),
                torch.zeros(1, batch_size, self.hidden))

In [16]:
import torch.optim as optim

# Hyperparameters
hidden_size = 128
num_layers = 1
learning_rate = 0.001
num_epochs = 500

model = TextGenerationModel(len(chars), hidden_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    hidden = model.init_hidden(1)
    optimizer.zero_grad()

    input_seq = torch.eye(len(chars))[input_text[:-1]].unsqueeze(0)
    target_seq = torch.tensor(input_text[1:])

    output, hidden = model(input_seq, hidden)
    loss = criterion(output.squeeze(0), target_seq)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 3.2147488594055176
Epoch 50, Loss: 2.6612002849578857
Epoch 100, Loss: 1.6842775344848633
Epoch 150, Loss: 1.0095326900482178
Epoch 200, Loss: 0.6186106204986572
Epoch 250, Loss: 0.36406460404396057
Epoch 300, Loss: 0.21476125717163086
Epoch 350, Loss: 0.1237429827451706
Epoch 400, Loss: 0.07479315251111984
Epoch 450, Loss: 0.048585195094347


In [17]:
start_str = "Hello"
num_generate = 100

input_seq = torch.tensor([char_to_idx[c] for c in start_str])
input_seq = torch.eye(len(chars))[input_seq].unsqueeze(0)

hidden = model.init_hidden(1)
generated_text = start_str

for _ in range(num_generate):
    output, hidden = model(input_seq, hidden)
    
    # output: [1, sequence_length, vocab_size], we want the last character's output
    last_output = output[:, -1, :]  # Extract the last output in the sequence
    
    # Get the index of the character with the highest score
    next_char_idx = torch.argmax(last_output).item()
    
    # Add the predicted character to the generated text
    generated_text += idx_to_char[next_char_idx]
    
    # Prepare the next input sequence with the newly predicted character
    input_seq = torch.eye(len(chars))[torch.tensor([next_char_idx])].unsqueeze(0)

print(generated_text)


Hello world! This is a simple text generation example using PyTorch.......llo world! This is a simple tex
